**project running on: C:\Users\Test\qbio\project**

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----        2023-05-24  오후 4:14                code
d----        2023-05-24  오후 4:25                data
-a---        2023-05-24  오후 3:42            267 read.me

At start, There is two initial idea for project

**project candidates**

1. Binding site frequency analysis on LIN28A

    = (Fig.2E) Normalized relative frequency of Watson-Crick pair between two bases flanking LIN28A binding sites.

2. LIN28A binding predictor

    = (Fig.S3F) Positional emission probabilities of hidden Markov model of LIN28A recognition motif.
    
    > using ML, as basically GLM and RF regression and compare them by validation(AUC) and the result.

**Project starting point**

Based on result of Guided mission 003 

https://github.com/WoobeenJeong/bioinfo1_jwb/blob/main/CoLab_TermProj_2023_3.ipynb

**Basic settings**
- Directory include whole RNA-seq data: 
    C:\Users\Test\qbio\project\data

- workspace setting:
    [ installing ]
    conda (already)
    mamba
    bedtools (already)
    bioawk (already)
    samtools (already)

    > conda install -p c:\ProgramData\Anaconda3 ipykernel --update-deps --force-reinstall
    > conda install -y bedtools bioawk samtools

At First, we need to check file is properly downloaded from google drive.

140aaf30bcb9276cc716f8699f04ddd6  CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  CLIP-35L33G.bam.bai
328883a73d507eafbf5b60bd6b906201  RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  RNA-siLuc.bam
43590fdc4d81905c0432e0d1cb8cfd5b  RNA-siLuc.bam.bai
5c08a9297307bc83259e658c4474f0cc  RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  RPF-siLuc.bam
dc24f69e8f571fc8be30f28ce5b84fcd  RPF-siLuc.bam.bai

> md5sum C:/Users/Test/qbio/project/data/* (-> git bash terminal on VScode)

In [1]:
import hashlib
import glob
import os

projectdb = 'C:/Users/Test/qbio/project/data/*'
files = glob.glob(projectdb)

for file in files:
    with open(file, 'rb') as f:
        data = f.read()
        md5_hash = hashlib.md5(data).hexdigest()
        filename = os.path.basename(file)
        print(f"{md5_hash}\t{filename}")


140aaf30bcb9276cc716f8699f04ddd6	CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd	CLIP-35L33G.bam.bai
e22f2adecc2da272b1297850ac463105	fivepcounts-RPF-siLuc.bed
ed47186ee44654792a8a130c695f3a22	gencode-exons-containing-startcodon.bed
3469e3e3e018302c5488adc052fd1e21	gencode-plusexon.gtf
b1c38579a9e8e0313c1508f02ce0afaf	gencode-start.gtf
2386750ffd7ddd7288bb821ca018069b	matches.txt
7c9528c08ed2f1ae6e2de599db1aa8fb	read-counts.txt.summary
328883a73d507eafbf5b60bd6b906201	RNA-control.bam
02073818e2f398a73c3b76e5169de1ca	RNA-control.bam.bai
fef112c727244060ea62d3f2564a07f6	RNA-siLin28a.bam.bai
43590fdc4d81905c0432e0d1cb8cfd5b	RNA-siLuc.bam.bai
a1bb3e29be412dfd7fd8d16b1b1acc4c	RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707	RPF-siLuc.bam
dc24f69e8f571fc8be30f28ce5b84fcd	RPF-siLuc.bam.bai


In [2]:
import os
os.chdir('C:/Users/Test/qbio/project/data')

import the tools at once!

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter


Previously in Misson 003, There is some code mistaken.

In [5]:
%cd C:/Users/Test/qbio/project/data

C:\Users\Test\qbio\project\data


In [18]:
gencode_name = ['Chrom', 'ID', 'Level', 'Start', 'End', 'Dot1', 'Strand', 'Dot2','Info' ]
gencode = pd.read_csv('gencode.gtf', sep='\t', comment='#', index_col=0, names=gencode_name)
gencode.head()

,ID,Level,Start,End,Dot1,Strand,Dot2,Info
Chrom,,,,,,,,
chr1,HAVANA,gene,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; gene_type ""TEC..."
chr1,HAVANA,transcript,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
chr1,HAVANA,exon,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
chr1,ENSEMBL,gene,3172239,3172348,.,+,.,"gene_id ""ENSMUSG00000064842.3""; gene_type ""snR..."
chr1,ENSEMBL,transcript,3172239,3172348,.,+,.,"gene_id ""ENSMUSG00000064842.3""; transcript_id ..."


in the code  
**.str.contains( ,case=False)**  
case=False means 'mirlet7g' only identically  
case=True means allowing 'mirlet7g' 'Mirlet7g' 'MIRLET7G'

In [19]:
mirlet7g = gencode[gencode['Info'].str.contains('mirlet7g', case=False)]
mirlet7g.head()

,ID,Level,Start,End,Dot1,Strand,Dot2,Info
Chrom,,,,,,,,
chr9,ENSEMBL,gene,106056039,106056126,.,+,.,"gene_id ""ENSMUSG00000065440.3""; gene_type ""miR..."
chr9,ENSEMBL,transcript,106056039,106056126,.,+,.,"gene_id ""ENSMUSG00000065440.3""; transcript_id ..."
chr9,ENSEMBL,exon,106056039,106056126,.,+,.,"gene_id ""ENSMUSG00000065440.3""; transcript_id ..."
